In [5]:
import numpy as np

def kodla(data, H):
  """
  LDPC kodlayıcı fonksiyonu.

  Args:
      data: Kodlanacak 16 bitlik veri.
      H: Parite kontrol matrisi.

  Returns:
      Kodlanmış veri.
  """

  k = len(data)
  n = H.shape[1]
  x = np.concatenate((data, np.zeros(n - k)))

  # Parite kontrol denklemlerini kullanarak kodlama
  for i in range(H.shape[0]):
    # H matrisindeki her satır için
    satir = H[i]
    # Matrisin ilgili sütunlarındaki verilerin toplamını hesaplama
    toplam = np.dot(satir, x)
    # Eğer toplam 1 ise, ilgili parite bitini değiştirme
    if toplam % 2 == 1:
      # **Düzeltme:** Son biti değil, H matrisindeki ilgili parite bitini değiştir.
      x[np.where(satir == 1)[0][0]] = 1 - x[np.where(satir == 1)[0][0]]

  return x

def geri_al(alınan_veri, H, max_iterasyon=50):
  """
  Belief Propagation (İnanç Yayılımı) algoritması kullanarak LDPC kod çözücü.

  Args:
      alınan_veri: Alınan kodlanmış veri.
      H: Parite kontrol matrisi.
      max_iterasyon: Maksimum iterasyon sayısı.

  Returns:
      Çözülmüş veri.
  """

  n = H.shape[1]
  x = np.copy(alınan_veri)

  # İteratif olarak mesajları ileterek kod çözme
  for _ in range(max_iterasyon):
    # Mesajları düğümler arasında iletmek için döngü
    for i in range(n):
      # Her bir veri düğümü için
      for j in range(H.shape[0]):
        # H matrisinde 1 olan satır ve sütun indekslerini bulma
        if H[j, i] == 1:
          # Düğümden gelen mesajı hesaplama
          mesaj = hesapla_mesaj(H, x, j, i)
          # Mesajı ilgili parite kontrol düğümüne gönderme
          x[i] = mesaj

    # Tüm mesajlar iletildikten sonra, en olası değerlere göre karar verme
    x = np.round(x)

  # Kodlanmış veriden orijinal veriyi çıkarma
  return x[:16]

def hesapla_mesaj(H, x, j, i):
  """
  Bir veri düğümünden bir parite kontrol düğümüne mesaj hesaplar.

  Args:
      H: Parite kontrol matrisi.
      x: Veri vektörü.
      j: Parite kontrol düğümü indeksi.
      i: Veri düğümü indeksi.

  Returns:
      Hesaplanan mesaj.
  """

  # Parite kontrol düğümüne bağlı diğer veri düğümlerinin indekslerini bulma
  diğer_düğümler = np.where(H[j] == 1)[0]
  diğer_düğümler = diğer_düğümler[diğer_düğümler != i]

  # Diğer veri düğümlerinden gelen mesajların toplamını hesaplama
  toplam = 0
  for k in diğer_düğümler:
    toplam += x[k]

  # Mesajı hesaplama
  mesaj = 1 - (toplam % 2)

  return mesaj

# 16 bitlik örnek veri
data = np.array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1])

# LDPC parite kontrol matrisi (daha büyük ve düşük yoğunluklu)
H = np.array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]])

# Veriyi kodlama
kodlanmış_veri = kodla(data, H)

# Gürültü ekleme (örnek olarak 2 bit değiştirilmiş)
gürültülü_veri = np.copy(kodlanmış_veri)
gürültülü_veri[2] = 1 - gürültülü_veri[2]
gürültülü_veri[5] = 1 - gürültülü_veri[5]

# Veriyi çözme
çözülmüş_veri = geri_al(gürültülü_veri, H, max_iterasyon=100) # İterasyon sayısını artırdık

print("Orijinal veri:", data)
print("Kodlanmış veri:", kodlanmış_veri)
print("Gürültülü veri:", gürültülü_veri)
print("Çözülmüş veri:", çözülmüş_veri)

Orijinal veri: [1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1]
Kodlanmış veri: [1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
Gürültülü veri: [1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.]
Çözülmüş veri: [0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1.]
